# Week 2: Identify Nearest Health Facilities

<span style="color:red">
**UPDATE**

Thank you for your analysis. Despite our warning efforts so far, the virus continues to spread rapidly. We want to get infected individuals treatment as quickly as possible, so we need your help to calculate which hospital or clinic is closest to each known infected individual in the population.
</span>

Your goal for this notebook will be to identify the nearest hospital or clinic for each infected person.

## Imports

In [42]:
import cudf
import cuml
import cupy as cp

## Load Population Data

Begin by loading the `lat`, `long` and `infected` columns from `'./data/week2.csv'` into a cuDF data frame called `gdf`.

In [43]:
gdf = cudf.read_csv("./data/week2.csv" , usecols=["lat","long","infected"])
gdf

,lat,long,infected
0,54.522510,-1.571896,0.0
1,54.554030,-1.524968,0.0
2,54.552486,-1.435203,0.0
3,54.537189,-1.566215,0.0
4,54.528212,-1.588462,0.0
...,...,...,...
58479889,51.634416,-2.925863,0.0
58479890,51.556972,-3.036290,0.0
58479891,51.588992,-2.921915,0.0
58479892,51.590974,-2.954539,0.0


## Load Hospital and Clinics Data

For this step, your goal is to create an `all_med` cuDF data frame that contains the latitudes and longitudes of all the hospitals (data found at `'./data/hospitals.csv'`) and clinics (data found at `'./data/clinics.csv'`).

In [44]:
hospitals = cudf.read_csv("./data/hospitals.csv", usecols=["Latitude","Longitude"])
clinics = cudf.read_csv("./data/clinics.csv", usecols=["Latitude","Longitude"])

frames = [hospitals,clinics]
all_med = cudf.concat(frames)

all_med = cudf.DataFrame({"lat" : all_med["Latitude"], "long" : all_med["Longitude"]})
all_med

,lat,long
0,51.379997,-0.406042
1,51.315132,-0.556289
2,51.437195,-2.847193
3,53.459743,-2.245469
4,52.078121,-0.030604
...,...,...
19077,53.737923,-1.767982
19078,51.341579,-2.784808
19079,51.286285,-2.820388
19080,53.163048,-2.205781


Since we will be using the coordinates of those facilities, keep only those rows that are non-null in both  `Latitude` and `Longitude`.

In [45]:
all_med = all_med.dropna()
all_med

,lat,long
0,51.379997,-0.406042
1,51.315132,-0.556289
2,51.437195,-2.847193
3,53.459743,-2.245469
4,52.078121,-0.030604
...,...,...
19077,53.737923,-1.767982
19078,51.341579,-2.784808
19079,51.286285,-2.820388
19080,53.163048,-2.205781


## Make Grid Coordinates for Medical Facilities

Provided for you in the next cell (which you can expand by clicking on the "...", and contract again after executing by clicking on the blue left border of the cell) is the lat/long to grid coordinates converter you have used earlier in the workshop. Use this converter to create grid coordinate values stored in `northing` and `easting` columns of the `all_med` data frame you created in the last step.

In [46]:
# https://www.ordnancesurvey.co.uk/docs/support/guide-coordinate-systems-great-britain.pdf

def latlong2osgbgrid_cupy(lat, long, input_degrees=True):
    '''
    Converts latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (N)
    long: longitude coordinate (E)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * cp.pi/180
        long = long * cp.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000 # northing of true origin
    E0 = 400000 # easting of true origin
    F0 = .9996012717 # scale factor on central meridian
    phi0 = 49 * cp.pi / 180 # latitude of true origin
    lambda0 = -2 * cp.pi / 180 # longitude of true origin and central meridian
    
    sinlat = cp.sin(lat)
    coslat = cp.cos(lat)
    tanlat = cp.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * cp.sin(latdiff) * cp.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * cp.sin(2*(latdiff)) * cp.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * cp.sin(3*(latdiff)) * cp.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - cp.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

In [47]:
cupy_lat = cp.asarray(all_med['lat'])
cupy_long = cp.asarray(all_med['long'])

northing, easting = latlong2osgbgrid_cupy(cupy_lat,cupy_long)

all_med['northing'] = northing.astype('float32')
all_med['easting'] = easting.astype('float32')

all_med

,lat,long,northing,easting
0,51.379997,-0.406042,165810.468750,510917.53125
1,51.315132,-0.556289,158381.343750,500604.84375
2,51.437195,-2.847193,171305.781250,341119.37500
3,53.459743,-2.245469,395944.562500,383703.59375
4,52.078121,-0.030604,244071.703125,534945.18750
...,...,...,...,...
19077,53.737923,-1.767982,426887.031250,415302.53125
19078,51.341579,-2.784808,160625.125000,345341.25000
19079,51.286285,-2.820388,154503.671875,342794.59375
19080,53.163048,-2.205781,362932.500000,386243.40625


## Find Closest Hospital or Clinic for Infected

Fit `cuml.NearestNeighbors` with `all_med`'s `northing` and `easting` values, using the named argument `n_neighbors` set to `1`, and save the model as `knn`.

In [48]:
knn = cuml.NearestNeighbors(n_neighbors=1)
knn.fit(all_med[['northing', 'easting']])

NearestNeighbors(n_neighbors=1, verbose=4, handle=<cuml.common.handle.Handle object at 0x7f87fd5424d0>, algorithm='brute', metric='euclidean', p=2, metric_params=None, output_type='cudf')

# Save every infected member in `gdf` into a new dataframe called `infected_gdf`.

In [49]:
infected_gdf = gdf[gdf["infected"] == 1].reset_index()
infected_gdf

,index,lat,long,infected
0,1346586,53.715826,-2.430079,1.0
1,1350932,53.664881,-2.425673,1.0
2,1352085,53.696765,-2.488940,1.0
3,1352799,53.696966,-2.488897,1.0
4,1357529,53.727804,-2.392959,1.0
...,...,...,...,...
70875,58459189,51.662717,-2.926850,1.0
70876,58459551,51.599350,-2.959175,1.0
70877,58461935,51.543825,-2.822984,1.0
70878,58464082,51.562536,-2.879492,1.0


Create `northing` and `easting` values for `infected_gdf`.

In [50]:
cupy_lat = cp.asarray(infected_gdf['lat'])
cupy_long = cp.asarray(infected_gdf['long'])

northing, easting = latlong2osgbgrid_cupy(cupy_lat,cupy_long)

infected_gdf['northing'] = northing.astype('float32')
infected_gdf['easting'] = easting.astype('float32')

infected_gdf

,index,lat,long,infected,northing,easting
0,1346586,53.715826,-2.430079,1.0,424489.781250,371619.68750
1,1350932,53.664881,-2.425673,1.0,418820.687500,371876.50000
2,1352085,53.696765,-2.488940,1.0,422394.406250,367721.00000
3,1352799,53.696966,-2.488897,1.0,422416.812500,367723.96875
4,1357529,53.727804,-2.392959,1.0,425808.125000,374076.56250
...,...,...,...,...,...,...
70875,58459189,51.662717,-2.926850,1.0,196451.125000,335900.96875
70876,58459551,51.599350,-2.959175,1.0,189433.453125,333573.00000
70877,58461935,51.543825,-2.822984,1.0,183143.984375,342935.12500
70878,58464082,51.562536,-2.879492,1.0,185270.296875,339042.06250


Use `knn.kneighbors` with `n_neighbors=1` on `infected_gdf`'s `northing` and `easting` values. Save the return values in `distances` and `indices`.

In [51]:
distances, indices = knn.kneighbors(infected_gdf[["northing","easting"]], n_neighbors=1)

### Check Your Solution

`indices`, returned from your use of `knn.kneighbors` immediately above, should map person indices to their closest clinic/hospital indices:

In [57]:
indices.head()

0    18316
1    12816
2     4489
3     4489
4     4962
dtype: int64

Here you can print an infected individual's coordinates from `infected_gdf`:

In [60]:
infected_gdf.iloc[0] # get the coords of an infected individual (in this case, individual 0)

index       1.346586e+06
lat         5.371583e+01
long       -2.430079e+00
infected    1.000000e+00
northing    4.244898e+05
easting     3.716197e+05
Name: 0, dtype: float64

You should be able to used the mapped index for the nearest facility to see that indeed the nearest facility is at a nearby coordinate:

In [62]:
all_med.iloc[18316] # printing the entry for facility 1234 (replace with the index identified as closest to the individual)

lat             53.733227
long            -2.455536
northing    426435.968750
easting     369952.218750
Name: 17095, dtype: float64

<div align="center"><h2>Please Restart the Kernel</h2></div>

...before moving to the next notebook.

In [63]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}